In [73]:
!pip install --quiet -U langchain langchain-community langchain-aws langchain-core

import os
import pandas as pd
import numpy as np
import json
import psycopg2
import ast
import pgvector
import math
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector
import boto3
from langchain_community.embeddings.bedrock import BedrockEmbeddings
from langchain_aws import BedrockLLM
import time

print("Imported and installed dependencies!")

Imported and installed dependencies!


For now, we are going to simply read the CSV file on the server. Look at other notebook on how to get CSV file from S3 Bucket.

In [ ]:
# Load your CSV file into a pandas DataFrame
df = pd.read_csv('whatever-the-CSV-file-name-is.csv')
df.head()

Need to create our own CSV file with URL and text columns only.

In [4]:
# Select only the 'url' and 'text' columns
selected_columns_df = df[['url', 'text']

# Define the path for the new CSV file
new_file_path = 'moded.csv'

# Check if the file already exists
if not os.path.exists(new_file_path):
    # Save the new CSV file
    selected_columns_df.to_csv(new_file_path, index=False)
    print(f"New CSV file created at: {new_file_path}")
else:
    print(f"CSV file already exists at: {new_file_path}")

CSV file already exists at: moded.csv


For each row of the moded.csv file, we will create vector embeddings using the Amazon Titan Embeddings model.

In [5]:
if not os.path.exists('moded_with_embeddings.csv'):
    # Load the CSV file
    file_path = 'moded.csv'
    data = pd.read_csv(file_path)

    # Initialize the Bedrock Embeddings model
    embeddings = BedrockEmbeddings()

    # Initialize a list to store the embeddings
    embeddings_list = []

    # Iterate over each row in the DataFrame
    for index, row in data.iterrows():
        text = row['text']
        if pd.isna(text):
            text = " "
        embedding = embeddings.embed_query(text)
        embeddings_list.append(embedding)
        if (index + 1) % 500 == 0:
            print(f"Processed {index + 1}/{len(data)} rows")

    # Add the embeddings to the DataFrame
    data['embedding'] = embeddings_list

    # Save the updated DataFrame to a new CSV file
    data.to_csv('moded_with_embeddings.csv', index=False)
else:
    print("moded_with_embeddings already exists")

moded_with_embeddings already exists


Sanity check to see if CSV file with embeddings has correct data.

In [6]:
# Load the CSV file with embeddings
file_path_with_embeddings = 'moded_with_embeddings.csv'
data_with_embeddings = pd.read_csv(file_path_with_embeddings)

# Perform the sanity check to count the number of rows
number_of_rows = len(data_with_embeddings)

print(f"The number of rows in the 'moded_with_embeddings.csv' file is: {number_of_rows}")

# Print the first row of the CSV file
first_row = data_with_embeddings.iloc[0]
print("The first row in the 'moded_with_embeddings.csv' file is:")
print(first_row.to_dict())

The number of rows in the 'moded_with_embeddings.csv' file is: 15472
The first row in the 'moded_with_embeddings.csv' file is:
{'url': 'https://vancouver.calendar.ubc.ca', 'text': nan, 'embedding': '[0.22460938, -0.3046875, -0.12695312, 0.27539062, 0.0021972656, -0.31835938, 0.86328125, -0.0018081665, -0.09863281, 0.13378906, 0.40625, 0.23632812, -0.19726562, 0.13378906, 0.59375, 0.46289062, 0.44726562, 0.23632812, -1.4921875, 0.5703125, 0.041015625, 0.04272461, -0.009643555, 0.95703125, -0.33398438, -0.30078125, -0.7578125, -0.6484375, -0.6953125, -0.33984375, -0.3515625, 0.51171875, 0.09765625, -0.70703125, 0.27929688, 0.10205078, -0.1875, -0.79296875, 0.16894531, -0.62109375, -0.22851562, 0.1640625, 0.21777344, -0.68359375, 0.17773438, 0.6640625, -0.6328125, 1.421875, 1.0234375, -0.32421875, -0.2109375, -0.5546875, -0.80078125, 0.78515625, 0.42578125, 0.03515625, 0.28320312, 0.9375, 1.2890625, -0.068847656, -0.5078125, 0.47851562, 0.068847656, 0.65625, 0.025878906, 0.14746094, 0.925

Connect to RDS instance and install vector extension if not already there.

In [ ]:
# Define the connection parameters
connection_params = {
    "your-database-parameters"
}

# Create the connection string
connection_string = " ".join([f"{key}={value}" for key, value in connection_params.items()])

print("Connection string:", connection_string)

# Connect to PostgreSQL database in Timescale using connection string
conn = psycopg2.connect(connection_string)
cur = conn.cursor()

#install pgvector
cur.execute("CREATE EXTENSION IF NOT EXISTS vector");
conn.commit()

print("Connected to RDS instance!")

Registers the vector type with psycopg2, enabling the handling of vector data types within Python.

In [8]:
# Register the vector type with psycopg2
register_vector(conn)

Create our vector table.

In [9]:
# Create table to store embeddings and metadata
table_create_command = """
CREATE TABLE IF NOT EXISTS test_embeddings (
            id bigserial primary key,
            url text,
            text text,
            embedding vector(1536)
            );
            """

cur.execute(table_create_command)
cur.close()
conn.commit()

Populate test_embeddings table.

In [10]:
# Load the CSV file with embeddings
print("Loading the CSV file...")
file_path_with_embeddings = 'moded_with_embeddings.csv'
data_with_embeddings = pd.read_csv(file_path_with_embeddings)

# Check the type of the embedding column
embedding_type = type(first_row['embedding'])
print(f"The type of the embedding column is: {embedding_type}")

# Function to convert string representation of list to numpy array
def parse_embedding(embedding_str):
    return np.array(eval(embedding_str))

# Apply the function to the embedding column if it's a string
if isinstance(first_row['embedding'], str):
    print("Converting the embedding column to numpy arrays...")
    data_with_embeddings['embedding'] = data_with_embeddings['embedding'].apply(parse_embedding)
    print("Conversion complete.")
else:
    print("Embeddings are not strings, they are a list of floats")

# Verify the conversion
first_row_converted = data_with_embeddings.iloc[0]
print("The first row after converting the 'embedding' column:")
print(first_row_converted.to_dict())

# Prepare the list of tuples to insert
print("Preparing the list of tuples for insertion...")
data_list = [(row['url'], row['text'], row['embedding']) for index, row in data_with_embeddings.iterrows()]
print("Preparation complete.")

# Use execute_values to perform batch insertion
print("Performing batch insertion...")
cur = conn.cursor()
execute_values(cur, "INSERT INTO test_embeddings (url, text, embedding) VALUES %s", data_list)

# Commit after we insert all embeddings
print("Committing the transaction...")
conn.commit()

print("Batch insertion complete!")

Loading the CSV file...
The type of the embedding column is: <class 'str'>
Converting the embedding column to numpy arrays...
Conversion complete.
The first row after converting the 'embedding' column:
{'url': 'https://vancouver.calendar.ubc.ca', 'text': nan, 'embedding': array([ 0.22460938, -0.3046875 , -0.12695312, ...,  0.05712891,
        0.13769531, -1.0390625 ])}
Preparing the list of tuples for insertion...
Preparation complete.
Performing batch insertion...
Committing the transaction...
Batch insertion complete!


Perform sanity checks on embeddings table.

In [107]:
cur = conn.cursor()
num_records = 0
try:
    cur.execute("SELECT COUNT(*) as cnt FROM test_embeddings;")
    num_records = cur.fetchone()[0]
    print("Number of vector records in table: ", num_records,"\n")
except:
    print("Error when counting number of rows in embeddings table!")
    conn.rollback()
finally:
    cur.close()

Number of vector records in table:  15472 



In [108]:
cur = conn.cursor()
try:
    # print the first record in the table, for sanity-checking
    cur.execute("SELECT * FROM test_embeddings LIMIT 1;")
    records = cur.fetchall()
    print("First record in table: ", records)
except:
    print("Error when printing first row in embeddings table!")
    conn.rollback()
finally:
    cur.close()

First record in table:  [(1, 'https://vancouver.calendar.ubc.ca', 'NaN', array([ 0.22460938, -0.3046875 , -0.12695312, ...,  0.05712891,
        0.13769531, -1.0390625 ], dtype=float32))]


Apply Indexing

In [135]:
# Drops existing indexes on the embedding column
def drop_existing_indexes():
    try:
        cur = conn.cursor()
        cur.execute("""
            SELECT indexname 
            FROM pg_indexes 
            WHERE tablename = 'test_embeddings' 
            AND indexdef LIKE '%embedding%' 
            AND indexdef NOT LIKE '%pkey%';
        """)
        indexes = cur.fetchall()
        for index in indexes:
            cur.execute(f"DROP INDEX IF EXISTS {index[0]};")
        conn.commit()
        print("Dropped existing indexes on embedding column!")
    except Exception as e:
        print("Error dropping existing indexes:", e)
        conn.rollback()
    finally:
        cur.close()

# Create an index on the data for faster retrieval
def create_index(index_method, distance_measure):
    drop_existing_indexes()
    try:
        cur = conn.cursor()
        if index_method == 'hnsw':
            cur.execute(f'CREATE INDEX ON test_embeddings USING hnsw (embedding {distance_measure})')
        elif index_method == 'ivfflat':
            num_lists = num_records / 1000
            if num_lists < 10:
                num_lists = 10
            if num_records > 1000000:
                num_lists = math.sqrt(num_records)

            cur.execute(f'CREATE INDEX ON test_embeddings USING ivfflat (embedding {distance_measure}) WITH (lists = {num_lists});')

        conn.commit()
        print("Created Index!")
    except:
        print("Error when indexing embeddings table!")
        conn.rollback()
    finally:
        cur.close()

create_index('hnsw', 'vector_l2_ops')

Dropped existing indexes on embedding column!
Created Index!


Sanity Check to see if index is on embeddings table.

In [136]:
# Perform sanity check to print all indexes on test_embeddings
try:
    cur = conn.cursor()
    cur.execute("SELECT indexname FROM pg_indexes WHERE tablename = 'test_embeddings';")
    indexes = cur.fetchall()
    print("Indexes on test_embeddings table:")
    for index in indexes:
        print(index[0])
except Exception as e:
    print("Error during sanity check:", e)
finally:
    cur.close()

Indexes on test_embeddings table:
test_embeddings_embedding_idx
test_embeddings_pkey


Sanity Check to see details of index on embeddings table.

In [140]:
# Function to get details of an index
def get_index_details(index_name):
    index_details_query = f"""
    SELECT indexname, indexdef
    FROM pg_indexes
    WHERE indexname = '{index_name}';
    """

    try:
        cur = conn.cursor()
        cur.execute(index_details_query)
        index_details = cur.fetchone()
        if index_details:
            print(f"Details of index '{index_name}':")
            print(f"Index Name: {index_details[0]}")
            print(f"Index Definition: {index_details[1]}")
        else:
            print(f"No details found for index '{index_name}'.")
    except:
        print("Error when checking indexing details!")
        conn.rollback()
    finally:
        cur.close()

# Verify details of the created index
get_index_details('test_embeddings_embedding_idx')

Details of index 'test_embeddings_embedding_idx':
Index Name: test_embeddings_embedding_idx
Index Definition: CREATE INDEX test_embeddings_embedding_idx ON public.test_embeddings USING hnsw (embedding vector_l2_ops)


Retrieval Step.

In [141]:
# Get most similar documents from the database
def get_docs(query_embedding, number):
    embedding_array = np.array(query_embedding)
    # Register pgvector extension
    register_vector(conn)
    top_docs = []
    try:
        cur = conn.cursor()
        # Get the top N most similar documents using the KNN <=> operator
        cur.execute(f"SELECT url, text FROM test_embeddings ORDER BY embedding <=> %s LIMIT {number}", (embedding_array,))
        results = cur.fetchall()
        for result in results:
            doc_dict = {"url": result[0], "text": result[1]}
            top_docs.append(doc_dict)
        cur.close()
    except:
        print("Error when retrieving!")
        conn.rollback()
    finally:
        cur.close()
    return top_docs

# Initialize the Bedrock Embeddings model
embeddings = BedrockEmbeddings()

text = "Does physics 100 count for the arts requirement?"
docs = get_docs(embeddings.embed_query(text), 5)
for idx, doc in enumerate(docs, 1):
    print(f"\nDocument {idx}:\n{doc['url']}\n{doc['text']}")


Document 1:
https://vancouver.calendar.ubc.ca/faculties-colleges-and-schools/faculty-science/bachelor-science/microbiology-and-immunology#5622
Students requiring PHYS 100 may delay other 100-level PHYS courses until second year. ): 3
  - Communication Requirement ( A total of 6 credits of coursework is required to meet the Communication Requirement. For a full list of acceptable courses see Communication Requirement. ): 3
  - DSCI 100: 3
  - Electives ( Elective credits together with required courses must fulfill the Faculty of Science’s:  
a) Foundational Requirement;  
b) Laboratory Science Requirement;  
c) Science Breadth Requirement;  
d) Science and Arts Requirements;  
e) Upper-level Requirement;  
f) General Degree Requirements. In addition to meeting the minimum Upper-level Requirement (footnote 6), students must select an additional 12 credits of upper-level courses.

Document 2:
https://vancouver.calendar.ubc.ca/faculties-colleges-and-schools/faculty-science/bachelor-scienc

Putting everything together.

In [146]:
# Defining Constants
LLAMA_3_8B = "meta.llama3-8b-instruct-v1:0"
LLAMA_3_70B = "meta.llama3-70b-instruct-v1:0"
MISTRAL_7B = "mistral.mistral-7b-instruct-v0:2"
MISTRAL_LARGE = "mistral.mistral-large-2402-v1:0"

# Format all texts in the doc as one string when we pass prompt to LLM
def format_docs(docs):
    formatted_docs = "\n".join([f"Document {idx}:\n{doc['text']}" for idx, doc in enumerate(docs, 1)])
    return formatted_docs

# Get most similar documents from the database
def get_docs(query_embedding, number):
    embedding_array = np.array(query_embedding)
    # Register pgvector extension
    register_vector(conn)
    top_docs = []
    try:
        cur = conn.cursor()
        # Get the top N most similar documents using the KNN <=> operator
        cur.execute(f"SELECT url, text FROM test_embeddings ORDER BY embedding <=> %s LIMIT {number}", (embedding_array,))
        results = cur.fetchall()
        # Each item in list will be a dictionary with key values 'url' and 'text'
        for result in results:
            doc_dict = {"url": result[0], "text": result[1]}
            top_docs.append(doc_dict)
        cur.close()
    except:
        print("Error when retrieving!")
        conn.rollback()
    finally:
        cur.close()
    return top_docs

def check_if_documents_relates(docs, user_prompt, llm):

    system_prompt = "You are tasked with determining if the document helps answer the question. Provide a yes or no answer with a short explaination why. Avoid any unrelated information or questions."

    doc_relates = []
    for doc in docs:
        if llm.model_id == LLAMA_3_8B or llm.model_id == LLAMA_3_70B:
            prompt = f"""
                <|begin_of_text|>
                <|start_header_id|>system<|end_header_id|>
                {system_prompt}
                <|eot_id|>
                <|start_header_id|>question<|end_header_id|>
                {user_prompt}
                <|eot_id|>
                <|start_header_id|>document<|end_header_id|>
                {doc['text']}
                <|eot_id|>
                <|start_header_id|>assistant<|end_header_id|>
                """
        else:
            prompt = f"""Here is a queston that a user asked: {user_prompt}.
                Here is the text from a document: {doc['text']}.
                {system_prompt}
                """
        response = llm.invoke(prompt)
        doc_relates_dict = {"url": doc['url'], "text": doc['text'], "relate": response}
        doc_relates.append(doc_relates_dict)

    return doc_relates

def answer_prompt(user_prompt, number_of_docs):

    # Record the start times
    total_start_time = time.time()
    answer_start_time = time.time()

    # Initialize the Bedrock Embeddings model
    embeddings = BedrockEmbeddings()

    docs = get_docs(embeddings.embed_query(user_prompt), number_of_docs)

    documents = format_docs(docs)

    # Get the LLM we want to invoke
    llm = BedrockLLM(
                        model_id=LLAMA_3_8B
                    )

    system_prompt = "You are a helpful UBC student advising assistant who answers with kindness while being concise. Only generate one human readable answer"

    if llm.model_id == LLAMA_3_8B or llm.model_id == LLAMA_3_70B:
        prompt = f"""
            <|begin_of_text|>
            <|start_header_id|>system<|end_header_id|>
            {system_prompt}
            <|eot_id|>
            <|start_header_id|>user<|end_header_id|>
            {user_prompt}
            <|eot_id|>
            <|start_header_id|>documents<|end_header_id|>
            {documents}
            <|eot_id|>
            <|start_header_id|>assistant<|end_header_id|>
            """
    else:
        prompt = f"""{system_prompt}.
            Here is the question: {user_prompt}.
            Here are the source documents: {documents}
            """

    answer = llm.invoke(prompt)

    # Record the end time and find duration of answer only
    answer_end_time = time.time()
    answer_duration = answer_end_time - answer_start_time

    check_docs = check_if_documents_relates(docs, user_prompt, llm)

    # Record the end time and find duration of the total time of checking over each document
    total_end_time = time.time()
    total_duration = total_end_time - total_start_time

    return {"answer": answer, "check_docs": check_docs, "answer_time": answer_duration, "total_time": total_duration}

# Neatly prints dictionary returned by answer_prompt
def neat_print(response):
    print(f"answer: {response['answer']}\n")
    print("check_docs:")
    for doc in response['check_docs']:
        print(f"\nURL: {doc['url']}\nText:\n{doc['text']}\nRelevance: {doc['relate']}\n")

    print(f"answer_time: {response['answer_time']}\n")
    print(f"total_time: {response['total_time']}\n")

response = answer_prompt("Does physics 100 count for the arts requirement?", 5)

neat_print(response)

answer:  Hi there! Unfortunately, PHYS 100 does not count towards the Arts requirement. According to the documents, PHYS 100 is a foundational course for Physics students and is not considered an Arts course. To fulfill the Arts requirement, you'll need to take courses from the Faculty of Arts, excluding certain courses like GEOS, PSYC, and some others. If you have any more questions or need help with course planning, feel free to ask!

check_docs:

URL: https://vancouver.calendar.ubc.ca/faculties-colleges-and-schools/faculty-science/bachelor-science/microbiology-and-immunology#5622
Text:
Students requiring PHYS 100 may delay other 100-level PHYS courses until second year. ): 3
  - Communication Requirement ( A total of 6 credits of coursework is required to meet the Communication Requirement. For a full list of acceptable courses see Communication Requirement. ): 3
  - DSCI 100: 3
  - Electives ( Elective credits together with required courses must fulfill the Faculty of Science’s:  


In [129]:
response = answer_prompt("Can I add a minor in fourth year?", 5)

neat_print(response)

answer:  Hi there! I'm happy to help you with your question. According to the documents I've found, it's possible to add a minor in fourth year, but it's essential to plan carefully and ensure that the courses fit into your program timetable. You should also be aware of the prerequisites for the upper-level courses and the limited space in some courses. Additionally, you may need to complete an additional term or terms beyond the four years required for your degree. I recommend consulting with a departmental advisor in Science and a senior advisor in your home faculty to discuss your options and ensure a smooth transition.

check_docs:

URL: https://vancouver.calendar.ubc.ca/faculties-colleges-and-schools/faculty-forestry/buf-bachelor-urban-forestry/introduction
Text:
Students who choose not to select a minor will make up the 18 credits that comprise a minor through a selection of courses from both minor pools. Students apply to the program director for entry into one of the two minors

In [130]:
response = answer_prompt("What are all the specializations in the Faculty of Science?", 5)

neat_print(response)

answer:  The Faculty of Science at UBC offers a wide range of specializations, including:

* Astronomy
* Biochemistry
* Biological Sciences
* Botany
* Chemistry
* Computer Science
* Earth and Ocean Sciences
* Environmental Science
* Geology
* Kinesiology
* Mathematics
* Microbiology
* Physics
* Psychology
* Statistics

Please note that this is not an exhaustive list, and new specializations may be added or updated. I recommend visiting the UBC Academic Calendar or consulting with a program advisor to get the most up-to-date information on the specializations available.

check_docs:

URL: https://science.ubc.ca/students/first-year-courses
Text:


 

UBC Science offers numerous areas of specializations for students. Visit the UBC Academic Calendar and scroll down to check out the details of your interested specialization(s). Each specialization Calendar entry will show you an overview of your degree, including the sample timetable you could have at each class standing.

Relevance:  No. T